In [50]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import json
from pandas.io.json import json_normalize

import datetime

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
import os
os.listdir("input/")

['.DS_Store', 'test_cleaned.csv', 'train_cleaned.csv', 'sample_submission.csv']

In [3]:
train = pd.read_csv("input/train.csv", dtype={"fullVisitorId": "str"})
test = pd.read_csv("input/test.csv", dtype={"fullVisitorId": "str"})

FileNotFoundError: File b'input/train.csv' does not exist

In [ ]:
train.head()

In [ ]:
type(train.fullVisitorId[0])

In [ ]:
columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
dt_test = train.head(5)
dt_test

In [ ]:
dt_test.shape

In [ ]:
def load_data(path, nrows=None):
    columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(
                        path, 
                        converters={column: json.loads for column in columns_json}, 
                        dtype={'fullVisitorId':'str'})
    for column in columns_json:
        df_columns_json = json_normalize(df[column])
        df_columns_json.columns = [f"{column}.{subcolumn}" for subcolumn in df_columns_json.columns]
        df = df.drop(column, axis=1).merge(df_columns_json, left_index=True, right_index=True)
    return df

In [ ]:
%%time
train = load_data("input/train.csv")
test = load_data("input/test.csv")

In [ ]:
train.to_csv("input/train_extend.csv", index=False)
test.to_csv("input/test_extend.csv", index=False)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

trainset have 55 column while having 53 in testset. We knew that totals.transactionRevenue in training set is our target. We should find a column different between two set and drop it in training set

In [ ]:
col_drop = set(test.columns).difference(set(train.columns))
print(col_drop)

train = train.drop(col_drop, axis=1)

In [ ]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)
print("target shape: ", target.shape)

## Try to predict log(transactionRevenue) by training model using lightgbm

In [ ]:
target = train['totals.transactionRevenue']
target = np.log1p(target.fillna(0).astype('float64'))

drop all of constants columns in training set

In [ ]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False) == 1]
const_cols

In [ ]:
train = train.drop(const_cols, axis=1)
test = test.drop(const_cols, axis=1)

In [51]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (903653, 35)
test shape:  (804684, 34)


### Checking missing value

In [5]:
train = pd.read_csv("input/train_cleaned.csv", dtype={'fullVisitorId':'str'})
test = pd.read_csv("input/test_cleaned.csv", dtype={'fullVisitorId':'str'})

In [6]:
test.isna().sum()

channelGrouping                                 0
date                                            0
fullVisitorId                                   0
sessionId                                       0
visitId                                         0
visitNumber                                     0
visitStartTime                                  0
device.browser                                  0
device.deviceCategory                           0
device.isMobile                                 0
device.operatingSystem                          0
geoNetwork.city                                 0
geoNetwork.continent                            0
geoNetwork.country                              0
geoNetwork.metro                                0
geoNetwork.networkDomain                        0
geoNetwork.region                               0
geoNetwork.subContinent                         0
totals.bounces                                  0
totals.hits                                     0


In [ ]:
def check_missing_dt(train, test):
    train = train.iloc[:,:-1] # the last columns is our target, we have done with this columns earlier step
    missing_train = train.isnull().sum().values # array of all number of missing value per column
    missing_test = test.isnull().sum().values
    missing_train_per = missing_train/train.shape[0] # calculate percentage of missing value on total samples
    missing_test_per = missing_test/test.shape[0]
    return pd.DataFrame({"missing value's percentage of training": missing_train_per, 
                        "missing value's percentage of testing": missing_test_per}, 
                        index=train.columns).sort_values(
                                by=["missing value's percentage of training", "missing value's percentage of testing"], 
                                ascending=False)

In [ ]:
missing_data = check_missing_dt(train, test)
missing_data

Just the way that i struggle to seperate categorical columns and numerical columns

In [ ]:
cat_cols = []
num_cols = []
for c in train.columns:
    try:
        train[c].astype(np.float64)
        num_cols.append(str(c))
    except ValueError:
        cat_cols.append(str(c))
print("categorical columns: ",cat_cols)
print()
print("numerical columns:",num_cols)

**As we see on the numerical columns, there are some of columns classify like a numerical but naturelly these columns treat to reponse Yes No question, these columns are:**
1. device.isMobile
2. trafficSource.adwordsClickInfo.isVideoAd
3. trafficSource.adwordsClickInfo.page
4. trafficSource.isTrueDirect

------------
* fullVisitorId
* sessionId
* VisitId

*SessionId* **and** *VisitId* **combine to** *fullVisitorId*, **but fullVisitorId is our target to predict transactionRevenue, it was be formated like string**

------------
**Because we will split data training based on time, so** *VisitStartTime* **will be drop from the numerical columns**

------------
**and finally,** *totals.transactionRevenue* **is our output, it should be split into seperated dataframe** ***target***


In [7]:
num_cols = ['visitNumber', 
            'visitStartTime', 
            'totals.bounces', 
            'totals.hits', 
            'totals.newVisits', 
            'totals.pageviews']
cat_cols = ['channelGrouping', 
            'device.browser', 
            'device.deviceCategory', 
            'device.operatingSystem', 
            'geoNetwork.city', 
            'geoNetwork.continent', 
            'geoNetwork.country', 
            'geoNetwork.metro', 
            'geoNetwork.networkDomain', 
            'geoNetwork.region', 
            'geoNetwork.subContinent', 
            'trafficSource.adContent', 
            'trafficSource.adwordsClickInfo.adNetworkType', 
            'trafficSource.adwordsClickInfo.gclId', 
            'trafficSource.adwordsClickInfo.slot', 
            'trafficSource.campaign', 
            'trafficSource.keyword', 
            'trafficSource.medium', 
            'trafficSource.referralPath', 
            'trafficSource.source', 
            'device.isMobile', 
            'trafficSource.adwordsClickInfo.isVideoAd', 
            'trafficSource.adwordsClickInfo.page', 
            'trafficSource.isTrueDirect']

In [8]:
print(len(num_cols)+ len(cat_cols))

30


    the function to plot the categorical columns

In [ ]:
device = train['device.browser'].value_counts()[:6]
print(device.values)
trace = go.Bar(
            x= device.index,
            y= device.values,
            showlegend=True,
            marker = dict(
                color='rgba(255,0,255,0.8)'
            )
)
iplot([trace], filename="Device Browser")

In [ ]:
def plot_cat(data, color, type_plot=None):
    if type_plot=='Bar':
        trace = go.Bar(
            x = data.index[::-1],
            y = data.values[::-1],
            showlegend=False,
            marker = dict(
                color=color
            )
        )
    elif type_plot=='Scatter':
        trace = go.Scatter(
            x = data.index[::-1],
            y = data.values[::-1],
            showlegend=False,
            marker = dict(
                color= color
            )
        )
    else:
        print("choose your plot type")
        return
    return trace

**Show 3 subcolumns of device column: **
* device.browser
* device.deviceCategory
* device.operatingSystem

> We explose each feature by our target that is totals.transactionRevenue

In [ ]:
#browser
browser = train.groupby('device.browser')['totals.transactionRevenue'].agg(['size','count'])
browser.columns = ["count", "count of non-zero revenue"]
browser = browser.sort_values(by="count", ascending=False)
trace1 = plot_cat(browser["count"].head(6), 'rgba(60,70,115,0.5)', 'Bar')
trace2 = plot_cat(browser["count of non-zero revenue"].head(6), 'rgba(210,157,37,0.9)', 'Bar')
#deviceCategory
category = train.groupby("device.deviceCategory")["totals.transactionRevenue"].agg(['size', 'count'])
category.columns = ["count", "count of non-zero revenue"]
category = category.sort_values(by="count", ascending=False)
trace3 = plot_cat(category["count"].head(8), 'rgba(155,155,13,0.5)', 'Bar')
trace4 = plot_cat(category["count of non-zero revenue"].head(8), 'rgba(210,30,222,0.5)', 'Bar')
#operatingSystem
operating = train.groupby("device.operatingSystem")["totals.transactionRevenue"].agg(['size', 'count'])
operating.columns = ["count", "count of non-zero revenue"]
operating = operating.sort_values(by="count", ascending=False)
trace5 = plot_cat(operating["count"].head(8), 'rgba(100,90,80,0.5)', 'Bar')
trace6 = plot_cat(operating["count of non-zero revenue"].head(8), 'rgba(60,99,123,0.5)', 'Bar')
fig = tools.make_subplots(rows=3, cols=2, subplot_titles=["Browser - count", "Browser - Non Zero Revenue",
                                                          "Category - count", "Category - Non Zero Revenue",
                                                          "Operating System - count", "Operating System - Non Zero Revenue"
                                                         ])
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 1,2)
fig.append_trace(trace3, 2,1)
fig.append_trace(trace4, 2,2)
fig.append_trace(trace5, 3,1)
fig.append_trace(trace6, 3,2)
fig['layout'].update(height=1300, width=900, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
iplot(fig, filename="device-plots")

In [9]:
train['date'] = train['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())

In [10]:
train['date'][0]

datetime.date(2016, 9, 2)

In [ ]:
date = train.groupby('date')['totals.transactionRevenue'].agg(['size', 'count'])
date.columns = ["count", "count of non-zero Revenue"]
date = date.sort_index()
trace1 = plot_cat(date["count"], 'red', 'Scatter')
trace2 = plot_cat(date["count of non-zero Revenue"], 'green', 'Scatter')
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=["count", "count of non zero revenue"])
fig['layout'].update(height=800, width=1000, title="Date plots", paper_bgcolor='rgba(233,233,233,0.4)')
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 2, 1)
iplot(fig, filename="date-plots")

In [ ]:
num_visit_per_id = train.groupby("fullVisitorId")["visitNumber"].agg(['size', 'count'])
num_visit_per_id.head()

In [ ]:
num_visit_per_id = train.groupby("fullVisitorId")["visitNumber"].agg(['size', 'count'])
num_visit_per_id.columns = ["count", "count of non zero revenue"]
num_visit_per_id.sort_values(by="count", ascending=False, inplace=True)
trace1 = plot_cat(num_visit_per_id["count"].head(20), 'rgba(145,210,50,0.5)', 'Bar')
trace2 = plot_cat(num_visit_per_id["count of non zero revenue"].head(20), 'rgba(145,210,50,0.5)', 'Bar')
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=["Count", "Count of non zero revenue"])
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 2,1)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgba(233,233,233,0.5)', title="Revenue per visitor id")
iplot(fig, filename="revenue-visitor")

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    lbd = LabelEncoder()
    lbd.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbd.transform(list(train[col].values.astype('str')))
    test[col] = lbd.transform(list(test[col].values.astype('str')))

In [ ]:
for col in num_cols:
    train[col] = train[col].fillna(0).astype('float')
    test[col] = test[col].fillna(0).astype('float')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train["totals.transactionRevenue"].fillna(0, inplace=True)
y_train = train["totals.transactionRevenue"].values
drop_col = set(train.columns).difference(set(cat_cols + num_cols))
drop_col = list(drop_col) + ["VisitStartTime"]
drop_col

In [ ]:
train['sessionId'].head()

In [ ]:
train['visitId'].head()

In [ ]:
train['fullVisitorId'].head()

In [ ]:
train['visitStartTime'].head()

In [ ]:
train['date'][0]

In [96]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

def run_lgb(X_train, y_train, X_val, y_val, X_test):
    data_train = lgb.Dataset(X_train, label=y_train)
    data_val = lgb.Dataset(X_val, label=y_val)
    param = {
         'objective':'regression',
         'metric': 'rmse',
         'learning_rate':0.005,
         'num_leaves':40,
         'min_data_in_leaf':150,
         'max_depth':10,
         'bagging_fraction':0.6,
         'feature_fraction':0.6,
         'bagging_frequency': 6,
         'verbosity':-1,
         'random_state': 42}
    model = lgb.train(param, data_train, valid_sets=[data_train, data_val], num_boost_round=5000, early_stopping_rounds=100,
                  verbose_eval=200)
    pred_y_test = model.predict(X_test, num_iteration=model.best_iteration)
    
    return pred_y_test, model

In [98]:
# split the train dataset into train and valid based on time
X_train_df = train[train["date"]<=datetime.date(2017,5,31)]
X_val_df = train[train["date"]>datetime.date(2017,5,31)]
y_train = np.log1p(X_train_df["totals.transactionRevenue"].values)
y_val = np.log1p(X_val_df["totals.transactionRevenue"].values)

In [91]:
y_train.shape

(608481,)

In [89]:
y_train = np.log1p(X_train_df[['fullVisitorId','totals.transactionRevenue']].groupby('fullVisitorId').sum().values.ravel())
X_train_df = X_train_df[cat_cols + num_cols + ['fullVisitorId']].groupby('fullVisitorId').mean().reset_index()
print(X_train_df.shape)
print(y_train.shape)

(608481, 31)
608481


In [92]:
y_val = np.log1p(X_val_df[['fullVisitorId','totals.transactionRevenue']].groupby('fullVisitorId').sum().values.ravel())
X_val_df = X_val_df[cat_cols + num_cols + ['fullVisitorId']].groupby('fullVisitorId').mean().reset_index()
print(X_val_df.shape)
print(len(y_val))

(110252, 31)
110252


In [73]:
test_df = test[cat_cols + num_cols + ['fullVisitorId']].groupby('fullVisitorId').mean().reset_index()
print(test_df.shape)

(617242, 31)


In [101]:
X_train = X_train_df[num_cols + cat_cols]
X_val = X_val_df[num_cols + cat_cols]
X_test = test[num_cols + cat_cols]

In [102]:
pred_test, model = run_lgb(X_train, y_train, X_val, y_val, X_test)

Training until validation scores don't improve for 100 rounds.
[200]	training's rmse: 0.28276	valid_1's rmse: 0.302188
[400]	training's rmse: 0.270983	valid_1's rmse: 0.288486
[600]	training's rmse: 0.265912	valid_1's rmse: 0.283481
[800]	training's rmse: 0.263015	valid_1's rmse: 0.281427
[1000]	training's rmse: 0.261247	valid_1's rmse: 0.280503
[1200]	training's rmse: 0.260001	valid_1's rmse: 0.280025
[1400]	training's rmse: 0.259134	valid_1's rmse: 0.279771
[1600]	training's rmse: 0.258435	valid_1's rmse: 0.279613
[1800]	training's rmse: 0.257823	valid_1's rmse: 0.279549
Early stopping, best iteration is:
[1866]	training's rmse: 0.257661	valid_1's rmse: 0.279545


In [103]:
test_df = test.copy()
test_df['predictions'] = pred_test

In [104]:
sub_pred_list = test_df[['fullVisitorId', 'predictions']].groupby('fullVisitorId')\
    .apply(lambda df: list(df.predictions))\
    .apply(lambda x: {'pred_'+str(i): pred for i, pred in enumerate(x)})

In [107]:
sub_data = test[num_cols + cat_cols + ['fullVisitorId']].groupby('fullVisitorId').mean()
sub_all_predictions = pd.DataFrame(list(sub_pred_list.values), index=sub_data.index)

In [109]:
sub_all_predictions.shape

(617242, 272)

In [17]:
#pred_y[pred_y<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":X_val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = X_val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_y)

val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), 
                                 np.log1p(val_pred_df["PredictedRevenue"].values))))

0.285425059978261


In [18]:
pred_test = model.predict(test[num_cols + cat_cols], num_iteration=model.best_iteration)

In [45]:
pred_test[pred_test<0]=0

In [40]:
submission = test[['fullVisitorId']].copy()

In [46]:
submission.loc[:,'PredictedLogRevenue'] = pred_test

In [47]:
grouped_test = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()

In [52]:
grouped_test.shape

(617242, 2)

In [49]:
grouped_test.to_csv("input/submit.csv", index=False)

In [ ]:
train.to_csv("input/train_work.csv", index=False)
test.to_csv("input/test_work.csv", index=False)
target.to_csv("input/target_work.csv", index=False)

In [ ]:
train = pd.read_csv("input/train_work.csv", dtype={'fullVisitorId':'str'})
test = pd.read_csv("input/test_work.csv", dtype={'fullVisitorId':'str'})
target = pd.read_csv("input/target_work.csv")

In [56]:
train.shape

(903653, 35)

In [53]:
test.shape

(804684, 34)

In [60]:
data_test = test[cat_cols + num_cols + ['fullVisitorId']].groupby('fullVisitorId').mean().reset_index()
data_test.head()

,fullVisitorId,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,...,device.isMobile,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.isTrueDirect,visitNumber,visitStartTime,totals.bounces,totals.hits,totals.newVisits,totals.pageviews
0,0000000259678714014,4.0,35.0,0.0,7.0,540.000000,2.0,218.0,100.000000,19362.5,...,0.0,1.0,11.0,0.5,1.5,1.511913e+09,0.0,9.5,0.500000,6.5
1,0000049363351866189,2.0,35.0,0.0,3.0,540.333333,3.0,93.0,40.666667,0.0,...,0.0,1.0,11.0,0.0,2.0,1.505813e+09,1.0,1.0,0.333333,1.0
2,0000053049821714864,1.0,73.0,1.0,23.0,955.000000,2.0,9.0,122.000000,29081.0,...,1.0,1.0,11.0,1.0,1.0,1.517179e+09,1.0,1.0,1.000000,1.0
3,0000059488412965267,2.0,35.0,1.0,1.0,182.000000,2.0,218.0,19.000000,0.0,...,1.0,1.0,11.0,0.0,2.0,1.519587e+09,1.0,1.0,0.333333,1.0
4,0000085840370633780,4.0,72.0,0.0,7.0,749.000000,2.0,218.0,100.000000,0.0,...,0.0,1.0,11.0,1.0,1.0,1.504919e+09,0.0,2.0,1.000000,2.0


In [57]:
data_train = train[cat_cols + num_cols + ['fullVisitorId']].groupby('fullVisitorId').mean()
data_train.shape

(714167, 30)

In [ ]:
pred_sub, model = run_lgb()

In [ ]:
# save the data Frame we working:
train.to_csv("input/train_cleaned.csv", index=False)
test.to_csv("input/test_cleaned.csv", index=False)

In [ ]:
test.columns

In [ ]:
submit = pd.read_csv("input/sample_submission.csv", header=0)

In [ ]:
submit.shape

In [ ]:
submit.columns

In [ ]:
set(submit['fullVisitorId']).intersection(set(test['fullVisitorId']))

In [ ]:
submit['fullVisitorId'].unique().sum()